# Create Heatmaps for significant PI3K Signaling genes

This notebook looks at the significant genes in at least one cancer. Pancancer heatmaps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Regulation of RAC1 activity Homo sapiens 351aa...,20/38,1.112561e-08,0.000002,0,0,3.895750,71.346824,DOCK6;ARHGAP9;VAV3;RASGRF2;RAP1GDS1;VAV1;DEF6;...,NCI-Nature_2016
1,Class I PI3K signaling events Homo sapiens 12b...,19/48,6.480058e-06,0.000677,0,0,2.929928,35.003214,HSP90AA1;SYK;PLEKHA1;PDPK1;PTEN;PIK3CD;PIK3R1;...,NCI-Nature_2016
2,ATR signaling pathway Homo sapiens 8991cbac-61...,15/39,8.941964e-05,0.006230,0,0,2.846894,26.539231,RFC5;RFC3;TIPIN;RFC4;MCM7;RFC2;PLK1;RPA1;FANCD...,NCI-Nature_2016
3,E2F transcription factor network Homo sapiens ...,22/72,1.368548e-04,0.007151,0,0,2.261699,20.121410,PRMT5;RANBP1;RRM1;CDKN1B;CBX5;RRM2;APAF1;HDAC1...,NCI-Nature_2016
4,BCR signaling pathway Homo sapiens acbf44e2-61...,20/64,1.935528e-04,0.008091,0,0,2.313101,19.776925,MAP4K1;MAP2K1;SYK;CSNK2A1;CHUK;PDPK1;PTEN;PIK3...,NCI-Nature_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 1
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Class I PI3K signaling events Homo sapiens 12b82bb7-6191-11e5-8ac5-06603eb7f303
total genes: 19


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

19

In [6]:
# Only include p-values < 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= 0.01]

In [7]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [8]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [9]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['ADAP1', 'SYK', 'PIK3R1'], dtype=object)

In [10]:
# Only include p-values < 0.01
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= 0.01]

In [11]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [12]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

In [13]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Class I PI3K signaling events Homo sapiens 12b...,10/48,3.923280e-08,0.000008,0,0,10.137875,172.888814,ZAP70;RAP1A;SYK;PIK3CA;PTEN;BLNK;PLCG2;BTK;PIK...,NCI-Nature_2016
1,BCR signaling pathway Homo sapiens acbf44e2-61...,10/64,6.779369e-07,0.000071,0,0,7.603406,108.000392,MAP4K1;PPP3CB;PTPRC;SYK;CHUK;PIK3CA;PTEN;BLNK;...,NCI-Nature_2016
2,ATR signaling pathway Homo sapiens 8991cbac-61...,8/39,1.046196e-06,0.000073,0,0,9.981908,137.454359,RFC5;RFC3;RFC4;MCM7;RFC2;FANCD2;TOPBP1;MCM2,NCI-Nature_2016
3,Aurora B signaling Homo sapiens 304a75af-618c-...,8/39,1.046196e-06,0.000055,0,0,9.981908,137.454359,NSUN2;NCL;NCAPG;NCAPD2;SMC4;NCAPH;NDC80;SMC2,NCI-Nature_2016
4,IL8- and CXCR2-mediated signaling events Homo ...,7/34,4.898355e-06,0.000205,0,0,10.018606,122.493600,GNG2;PRKCB;RAC2;ELMO1;DOCK2;PLCB2;GNAI2,NCI-Nature_2016


In [14]:
index2 = 0
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Class I PI3K signaling events Homo sapiens 12b82bb7-6191-11e5-8ac5-06603eb7f303
total genes: 10


In [15]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['PTEN', 'CYTH1', 'SYK', 'BTK', 'PLCG2', 'PIK3CD', 'RAP1A', 'BLNK',
       'PIK3CA', 'ZAP70'], dtype=object)

In [16]:
# Only include p-values < 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= 0.01]

In [17]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

In [18]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['SYK'], dtype=object)

In [19]:
# Only include p-values < 0.01
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= 0.01]

In [20]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [21]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')

['SYK']


2020-06-15 16:43:57,444 Warning: No enrich terms using library NCI-Nature_2016 when cutoff = 0.05


In [22]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Atypical NF-kappaB pathway Homo sapiens d2dd12...,1/17,0.00085,0.177646,0,0,1176.470588,8317.997700,SYK,NCI-Nature_2016
1,Osteopontin-mediated events Homo sapiens 94bf7...,1/31,0.00155,0.161972,0,0,645.161290,4173.884068,SYK,NCI-Nature_2016
2,Alpha-synuclein signaling Homo sapiens 588134b...,1/32,0.00160,0.111464,0,0,625.000000,4023.607133,SYK,NCI-Nature_2016
3,GMCSF-mediated signaling events Homo sapiens 0...,1/36,0.00180,0.094048,0,0,555.555556,3511.104256,SYK,NCI-Nature_2016
4,FAS (CD95) signaling pathway Homo sapiens 79cc...,1/38,0.00190,0.079419,0,0,526.315789,3297.852692,SYK,NCI-Nature_2016


In [23]:
i = 4
print(enr3.res2d.Term[i])
enr3.res2d.Genes[i]

FAS (CD95) signaling pathway Homo sapiens 79cc9c14-6192-11e5-8ac5-06603eb7f303


'SYK'

In [24]:
# Look at p-values 
certain_gene = 'SYK'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
499,SYK,0.000320,-0.697550,Gbm
11632,SYK,0.001919,-0.225323,Hnscc
23793,SYK,0.021461,-0.489000,Luad
36955,SYK,0.155625,-0.475100,Lscc
53051,SYK,0.905056,0.020400,Brca
53874,SYK,0.003901,0.405618,Ov
66600,SYK,0.085643,0.079000,En
81038,SYK,0.775873,0.034700,Colon
